Projet kaggle : Tweet Sentiment Extraction

Emma BEUZE, Nacim BOUIA, Antoine FRANCESCHI, Ayoub RHIM, Guillaume RUSSIAS

In [ ]:
#???
import jax
import jax.numpy as jnp
import gzip
import pickle

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
#besoin d'une version > à 1.2.0
pd.__version__

Il y a 4 types de colonnes :


*   textID - unique ID for each piece of text 
*   text - the text of the tweet 
*   sentiment - the general sentiment of the tweet        => il y a 3 sentiments : positive, neutral, negative
*   selected_text - [train only] the text that supports the tweet's sentiment 


**OBJECTIF :  examiner le sentiment donné pour un tweet donné et déterminer le mot ou la phrase qui le soutient le mieux.**






In [ ]:
os.chdir( os.getcwd() + "/donnees" )

Page qui résume comment utiliser les pandas : https://ledatascientist.com/manipulez-vos-donnees-avec-pandas/

In [ ]:
# Load the dataset
train_set = pd.read_csv(r"train.csv")
test_set = pd.read_csv(r"test.csv")

#print(train_set.shape)    => (27481, 4)  (textID, text, selected_text, sentiment )
#print(test_set.shape)     => (3534, 3)   (textID, text, sentiment)

train_data_text = train_set["text"]
train_data_sentiment = train_set["sentiment"]

N_train = train_data_text.shape[0]

train_data = [(train_data_text[k],train_data_sentiment[k]) for k in range(N_train)]
train_labels = train_set["selected_text"]


# check data makes sense

print("Exemple 1 :",train_data[1], "extrait choisi :", train_labels[1])
print("Exemple 2 :",train_data[59], "extrait choisi :", train_labels[59])
print("Exemple 3 :",train_data[750], "extrait choisi :", train_labels[750])

Dans le concours, la métrique à utiliser est le score de Jaccard : </br>
https://en.wikipedia.org/wiki/Jaccard_index </br>
https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50.


In [ ]:
def jaccard(str1, str2): 
    try:
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    except:
        print(str1, str2)

In [ ]:
for x in range(4,0,-1):
    print(x)

In [ ]:
def merge(L,i,j):
    acc = L[i]
    for k in range(i+1,j):
        acc = acc + " " + L[k]
    return acc

def extract_closest_substring(str1, str2):
    """ /!\ Pas opti du tout /!\ """
    split1 = str1.split(" ")
    N = len(split1)
    maxa = 0
    stra = str1
    for i in range(N):
        for j in range(i, N+1):
            acc = merge(split1,i,j)
            jacc = jaccard(acc, str2)
            if jacc >= maxa:
                maxa = jacc
                stra = acc
    return stra

str1 = "Dans le concours, la métrique à utiliser est le score de Jaccard"
str2 = "Apprenez à utiliser le score de Jaccard au lien suivant"

print(jaccard(str1,str2))
extract_closest_substring(str1, str2)

On définit une classe qui servira à la première approche de classifieur. L'objectif est de répliquer une méthode du plus proche voisin.

In [ ]:
def row_jac(row):
    return jaccard(row.text_x, row.text_y)

def row_extr(row):
    return extract_closest_substring(row.text_y, row.selected_text)

In [ ]:
class RetardedClassifier():
    def __init__(self, n = 2000):
        self._train_plus = None
        self._train_neg = None
        self._depth = n
    
    def fit(self, train_set):
        #positive train examples
        self._train_plus = train_set[train_set.sentiment == "positive"]
        #negative train examples
        self._train_neg = train_set[train_set.sentiment == "negative"]
    
    def sign_run(self,test_frame,sign):
        """ Does a branch run """
        if sign == "neutral":
            aux = test_frame[test_frame.sentiment == sign]
            aux["selected_text"] = aux.text
            return aux[["textID", "text", "selected_text", "sentiment"]]
        
        test_frame_plus = test_frame[test_frame.sentiment == sign]
        
        aux = self._train_plus.loc[0:self._depth,:].merge(test_frame_plus, how='cross')
        
        aux2 = aux
        aux2["score"] = aux.apply(row_jac, axis = 1) 
        
        aux3 = aux2.sort_values('score', ascending=False).drop_duplicates(['text_y'])
        
        aux4 = aux3
        aux4["new_extr"] = aux3.apply(row_extr, axis=1)
        aux4 = aux4.loc[:,["textID_y","text_y","new_extr","sentiment_y"]]
        
        aux5 = aux4.rename(columns = {"textID_y":"textID", "text_y":"text", "new_extr":"selected_text", "sentiment_y":"sentiment"})
        
        return aux5
    
    def __call__(self, test_frame):
        #positive run
        acc_pos = self.sign_run(test_frame, "positive")
        acc_neg = self.sign_run(test_frame, "negative")
        acc_neu = self.sign_run(test_frame, "neutral")
        
        return pd.concat([acc_pos, acc_neg, acc_neu])
        

In [ ]:
R = RetardedClassifier(500)
R.fit(train_set)
KK = R(test_set)
KK = KK.sort_index()
KK